In [9]:
# Install required libraries
%pip install groq python-dotenv --quiet

In [10]:
import os
from groq import Groq
from dotenv import load_dotenv

# Load .env if running locally
load_dotenv()

# For Colab: set your key manually like this:
os.environ["GROQ_API_KEY"] = "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" # <--- REPLACE WITH YOUR ACTUAL KEY

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found. Please set it.")

client = Groq(api_key=GROQ_API_KEY)

print("Groq client initialized ✅")

Groq client initialized ✅


In [11]:
MODEL_NAME = "llama-3.1-8b-instant"

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Senior Technical Support Engineer.
Be precise, code-focused, and analytical.
Explain errors clearly and provide step-by-step solutions.
Include example code if needed."""
    },
    "billing": {
        "system_prompt": """You are a Billing Support Specialist.
Be empathetic, professional, and policy-driven.
Explain refund rules clearly and guide users politely."""
    },
    "general": {
        "system_prompt": """You are a friendly customer support assistant.
Answer casually and helpfully."""
    },
    "tool": {
        "system_prompt": """You are a system that decides if external data is required.
If the user asks for live prices or real-time info, respond with: USE_TOOL"""
    }
}

In [13]:
def route_prompt(user_input):
    router_prompt = f"""
Classify this text into one of these categories:
[technical, billing, general, tool]

Return ONLY the category name. No explanation.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a strict classifier."},
            {"role": "user", "content": router_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()
    return category

In [14]:
def fetch_bitcoin_price():
    # Mock tool response
    return "The current price of Bitcoin is approximately $65,000 USD (mock data)."

In [15]:
def process_request(user_input):

    category = route_prompt(user_input)
    print(f"🔎 Routed to: {category}")

    # Tool handling
    if category == "tool":
        if "bitcoin" in user_input.lower():
            return fetch_bitcoin_price()
        else:
            return "Tool requested but not recognized."

    # Fallback safety
    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

In [16]:
print(process_request("My python script is throwing an IndexError on line 5."))

🔎 Routed to: technical
A `IndexError` typically occurs when you're trying to access an element at an index that does not exist in a list or other sequence.

To help you resolve the issue, I'll need more information. Please provide:

1. The code snippet (line 5) where the error is occurring.
2. The entire error message, including the line number where the error is happening.
3. The relevant code surrounding line 5.

Once I have this information, I'll be able to provide a more detailed explanation and a step-by-step solution to resolve the `IndexError`.

Here's a general template to help you identify the issue:

```python
# Assuming your_list is the sequence you're trying to access
your_list = ...  # Initialize your list here

# Code that's supposed to run
index = 5  # Replace with the actual index you're trying to access
try:
    element = your_list[index]
except IndexError as e:
    print(f"Error: {e}")
    # Now, let's print your_list to see its contents
    print(your_list)
```

This

In [17]:
print(process_request("I was charged twice for my subscription this month."))

🔎 Routed to: billing
I'm so sorry to hear that you were charged twice for your subscription this month. I'd be happy to help you resolve the issue.

To process a refund, I'll need to review our refund policy. Our policy states that we will refund any duplicate or incorrect charges within 30 days of the initial charge date. Since you've contacted us within a reasonable timeframe, we can proceed with processing a refund for the duplicate charge.

To confirm, I'll check the order details to ensure that the duplicate charge was indeed a mistake on our part. Can you please provide me with your account information and the order number associated with the duplicate charge? This will help me expedite the refund process.

Additionally, I'd like to let you know that we'll also be looking into why this occurred to prevent any future duplicate charges. Your account will be credited with the refund within 3-5 business days, and you'll receive an email notification once the credit has been applied.


In [18]:
print(process_request("Hey there, how are you?"))

🔎 Routed to: general
I'm doing great, thanks for asking. It's a lovely day to be helping people with their questions. What seems to be on your mind? Do you need some assistance with something?


In [19]:
print(process_request("What is the current price of Bitcoin?"))

🔎 Routed to: general
I'd be happy to help you with that! However, please note that cryptocurrency prices can fluctuate rapidly and may change rapidly. 

That being said, I can try to give you an idea of the current price of Bitcoin. But to get the most up-to-date information, I would recommend checking a reliable cryptocurrency website or platform such as CoinMarketCap, Coinbase, or Binance. 

If you'd like, I can try to look up the current price for you, but please keep in mind that it might not be entirely accurate by the time you see this response. 

Would you like me to do that anyway?
